In [22]:
import pandas as pd
import requests
import uuid
import json

def make_connection(language, fromScript, toScript):
    subscription_key = '1212d7d306fb4388a534e586691d0b57'
    endpoint = 'https://api.cognitive.microsofttranslator.com'
    path = '/transliterate?api-version=3.0'
    params = '&language='+language+'&fromScript='+fromScript+'&toScript='+toScript
    constructed_url = endpoint + path + params
    
    headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
    }
    
    return constructed_url, headers

def transliterate(url, headers, array):
    body_json = []
    for items in array:
        dict = {'Text':items}
        body_json.append(dict)

    request = requests.post(url, headers=headers, json=body_json)
    response = request.json()
    
    result = list() 
    for item in response:
        result.append(item['text'].title())
    
    return result

if __name__ == '__main__':
    df = pd.read_csv('surnames.csv', sep=',', na_values='')
    url, headers = make_connection(language='hi', fromScript='deva', toScript='latn')
    sur_eng = list()
    tmp = list()
    for idx, row in df.iterrows():
        print(idx, end='\r', flush=True)
        if(idx % 10 == 0 and len(tmp) != 0):
            sur_eng.extend(transliterate(url, headers, tmp))
            tmp = list()
            tmp.append(row[2])
        else:
            tmp.append(row[2])

    if(len(tmp) != 0):        
        sur_eng.extend(transliterate(url, headers, tmp))
    
    df['surname_eng'] = sur_eng
    
    df.to_csv(sep=',', path_or_buf='surnames_eng.csv', header=True, index=False)